In [9]:
import pandas as pd
import json

# Load Excel sheets
dashboard = pd.ExcelFile("data.xlsx")
data = pd.read_excel(dashboard, "Statistics")
aes = pd.read_excel(dashboard, 'AEs')
referrals = pd.read_excel(dashboard, 'Referrals')

partnerDistricts =pd.read_excel("partnerDistrict.xlsx")

# Fill missing values and convert types
data.fillna(0, inplace=True)
aes.fillna(0, inplace=True)
referrals.fillna(0, inplace=True)
try:
    if type(referrals['ReferrelRecordingYear'])!= int:
        referrals['ReferrelRecordingYear'] = 2025
except:
    pass
try:
    data['year'] = data['year'].astype(int)
    data['recordingMonth'] = data['recordingMonth'].astype(str)
    aes['AERecordingYear'] = aes['AERecordingYear'].astype(int)
    aes['AERecordingMonth'] = aes['AERecordingMonth'].astype(str)
    referrals['ReferrelRecordingYear'] = referrals['ReferrelRecordingYear'].astype(int)
    referrals['ReferrelRecordingMonth'] = referrals['ReferrelRecordingMonth'].astype(str)
except:
    pass



# Process rows and build JSON
def process_row(row):
    try:
        row_data = row.to_dict()
        if int(pd.to_numeric(row.get('totalAES', 0), errors='coerce')) > 0:
            recordedAEs = aes[
                (aes['AERecordingSite'] == row['facilityName']) &
                (aes['AERecordingMonth'] == row['recordingMonth']) &
                (aes['AERecordingYear'] == row['year'])
            ]
    
            recordedAEs = recordedAEs[recordedAEs['ae_classification']!=0]
    
            row_data['matchingAES'] = recordedAEs.to_dict(orient='records')
        else:
            row_data['matchingAES'] = []
                
        if int(pd.to_numeric(row.get('referralToOtherServices', 0), errors='coerce')) > 0:
            otherReferrals = referrals[
                (referrals['ReferrelRecordingSite'] == row['facilityName']) &
                (referrals['ReferrelRecordingMonth'] == row['recordingMonth']) &
                (referrals['ReferrelRecordingYear'] == row['year'])
            ]
    
            otherReferrals = otherReferrals[otherReferrals['services_referred']!=0]
            # print(otherReferrals)
            row_data['otherReferrals'] = otherReferrals.to_dict(orient='records')
        else:
            print("no data")
            row_data['otherReferrals'] = []
    
        return row_data
    except:
        pass

partner_lookup = dict(zip(partnerDistricts['DISTRICT'], partnerDistricts['PARTNER']))

data['Partner'] = data['District'].map(partner_lookup)


json_data = data.apply(process_row, axis=1).tolist()

# Save to JSON
output_file = "data.json"
with open(output_file, "w") as file:
    json.dump(json_data, file, indent=4)

# Print the JSON
print(json.dumps(json_data, indent=4))



no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data


In [10]:
data

,AERecordingMonth,AERecordingSite,AERecordingYear,District,LinkagesSelection,ReferrelRecordingMonth,ReferrelRecordingSite,ReferrelRecordingYear,Site_Name,TotalMCsBYMethod,...,totalhivNegativeNC,uncircumcisedClientsForHTS,uncircumcised_care,uncircumcised_prep,uncircumcised_srh,uncircumcised_sti,vmmc_number,year,facilityType,Partner
0,0,0,0,MBIRE,0.0,0,0,0,0,12,...,0,0.0,0.0,0.0,0.0,0.0,0,2025,0,ZACH
1,0,0,0,MARONDERA,0.0,0,0,0,0,30,...,0,0.0,0.0,0.0,0.0,0.0,0,2025,0,ZACH
2,0,0,0,MARONDERA,0.0,0,0,0,0,24,...,0,0.0,0.0,0.0,0.0,0.0,0,2025,0,ZACH
3,0,0,0,GURUVE,0.0,0,0,0,0,7,...,0,0.0,0.0,0.0,0.0,0.0,0,2025,0,ZICHIRE
4,0,0,0,GURUVE,0.0,0,0,0,0,9,...,0,0.0,0.0,0.0,0.0,0.0,0,2025,0,ZICHIRE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379,0,0,0,CHITUNGWIZA,0.0,0,0,0,0,233,...,0,0.0,0.0,0.0,0.0,0.0,0,2025,0,ZICHIRE
380,0,0,0,ZAKA,0.0,0,0,0,0,14,...,0,0.0,0.0,0.0,0.0,0.0,0,2025,0,ZACH
381,0,0,0,ZAKA,0.0,0,0,0,0,11,...,0,0.0,0.0,0.0,0.0,0.0,0,2025,0,ZACH
382,0,0,0,HURUNGWE,0.0,0,0,0,0,12,...,0,0.0,0.0,0.0,0.0,0.0,0,2025,0,ZACH
